## TBD

I made this notebook as a placeholder for my intention to create the following using __differential equations__:

1. Earth orbiting the sun with appropriate sphere surface rendering, starfield, etc.
1. Two-Body Problem Scenarios using the above
1. Three-Body Problem Scenarios using the above
1. Possibly some General Relativity scenarios